## 다음 카페 크롤링

In [1]:
# 라이브러리 불러오기
from selenium import webdriver # 브라우저 자동화를 위한 모듈
import requests # HTTP 요청을 보내기 위한 모듈
from bs4 import BeautifulSoup as BS # HTML 내용 파싱을 위한 모듈
import pandas as pd # 데이터 조작 및 분석을 모듈
import time # 코드 실행 속도 조절을 위한 모듈
import re # 정규 표현식 사용을 위한 모듈
import pickle # 파이썬 객체 직렬화를 위한 모듈
from selenium.webdriver.common.by import By # 다양한 방법으로 엘리먼트를 찾기 위한 모듈

In [3]:
# 크롬 드라이버 파일 경로 설정
file_path = "C:\\Users\\cho03\\Downloads\\세미나\\방학 세미나\\"
path = file_path + 'chromedriver.exe'

# 크롬 드라이버 실행
driver = webdriver.Chrome(path)

C:\Users\cho03\AppData\Local\Temp\ipykernel_28308\388304711.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [4]:
# 다음 카페 검색 페이지로 이동
driver.get('https://search.daum.net/search?w=fusion&col=cafe&q=%EB%85%B8%EC%9D%B8+%EB%B6%80%EC%96%91&DA=STC&sd=20180705000000&ed=20230705235959&period=u')

In [5]:
# 제목과 URL을 저장할 리스트 초기화
title_list = []
url_list = []

# 카페 글 크롤링
for j in range(0,14): # 페이지 수만큼 반복
    for i in range(1,16): # 페이지 수만큼 반복
        try:
            # 글의 제목과 URL 추출
            title = driver.find_element(By.XPATH,'//*[@id="twcColl"]/div[2]/c-container/c-card[{}]/div/c-doc-web/div/div[1]'.format(i)).text # 제목
            title2 = driver.find_element(By.XPATH, '//*[@id="twcColl"]/div[2]/c-container/c-card[{}]/div/c-doc-web/div/div[1]/c-title'.format(i))
            url = title2.get_attribute('data-href') # URL 추출
    
            title_list.append(title) # 제목 추가
            url_list.append(url) # URL 추가
        except:
            continue
        
    driver.find_element(By.CLASS_NAME, 'btn_next').click() # 다음 페이지로 이동

In [6]:
# 본문, 좋아요 수, 댓글 수, 댓글, 이미지 수, 영상 수를 저장할 리스트 초기화
new_doc = []
like_cnt = []
comment_cnt = []
comment_list = []
img_cnt = []
div_cnt = []

# 카페 글 크롤링
for i in range(len(url_list)):
    url_path = url_list[i] # URL 불러오기
    driver.switch_to.window(driver.window_handles[0]) # 첫 번째 탭으로 이동
    driver.execute_script("window.open('{}')".format(url_path)) # url 실행
    driver.switch_to.window(driver.window_handles[1]) # 두 번째 탭으로 이동
    time.sleep(1) # 1초 대기
    
    iframes = driver.find_elements(By.TAG_NAME, 'iframe')
    if len(iframes) > 0:
        driver.switch_to.frame(0)# 첫 번째 iframe으로 전환
        html = driver.page_source  # 현재 페이지의 HTML 코드를 가져옴
        soup = BS(html, 'html.parser')  # BeautifulSoup 객체를 생성
        
        try:
            # 카페 본문 추출
            a = soup.find('table', class_= 'protectTable').get_text().strip()
            
            like = soup.find('button', {'id': 'recommend-btn'}) # 좋아요 버튼 요소 검색
            b = int(like.find('span', class_='txt_num').get_text()) # 좋아요 수 추출
            
            comment = soup.find('button', {'id':'comment-btn'}) # 댓글 버튼 요소 검색
            c = int(comment.find('span', class_='txt_num').get_text()) # 댓글 수 추출
            
            
            commentlist = soup.find_all('span', class_='original_comment') # 댓글 요소들을 모두 검색
            d = "\n".join(item.get_text() for item in commentlist) # 추출한 댓글을 줄바꿈 문자로 구분하여 저장
            
            img = soup.find_all('img', class_='txc-image') # 이미지 요소들을 모두 검색
            e = len(img) # 이미지 개수 저장
            
            div = soup.find_all('div', class_='figure-video') # 영상 요소들을 모두 검색
            f = len(div) # 영상 개수 저장
                
        except:
            a = 'null'
            
        # 추출한 데이터를 각각의 리스트에 추가
        new_doc.append(a)
        like_cnt.append(b)
        comment_cnt.append(c)
        comment_list.append(d)
        img_cnt.append(e)
        div_cnt.append(f)
        driver.switch_to.default_content() # iframe에서 기본 컨텐츠로 전환
    
    else:
        new_doc.append(' ') # 본문이 없을 경우 빈 문자열 추가
        like_cnt.append(' ') # 좋아요 수가 없을 경우 빈 문자열 추가
        comment_cnt.append(' ') # 댓글 수가 없을 경우 빈 문자열 추가
        comment_list.append(' ') # 댓글이 없을 경우 빈 문자열 추가
        img_cnt.append(' ') # 이미지 수가 없을 경우 빈 문자열 추가
        div_cnt.append(' ') # 영상 수가 없을 경우 빈 문자열 추가
        
    driver.close()
    
    time.sleep(2) # 2초 대기
    print(i+1) # 진행상황 출력

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


In [7]:
# 크롤링 데이터를 데이터프레임으로 변환
raw_data = pd.DataFrame()
raw_data['title'] = title_list
raw_data['doc'] = new_doc
raw_data['like'] = like_cnt
raw_data['comment_cnt'] = comment_cnt
raw_data['comment_list'] = comment_list
raw_data['img'] = img_cnt
raw_data['div'] = div_cnt
raw_data['ch'] = 'daum'
raw_data['ch2'] = 'cafe'

In [8]:
# 데이터프레임을 pickle 파일로 저장
f = open(file_path + "daum_cafe.pkl", "wb")
pickle.dump(raw_data,f)

f.close()

In [9]:
# 크롬 드라이버 종료
driver.quit()

In [9]:
# 저장된 pickle 파일을 불러옴
f = open(file_path + "daum_cafe.pkl", "rb")
temp_file = pickle.load(f)
f.close()

temp_file

,title,doc,like,comment_cnt,comment_list,img,div,ch,ch2
0,생산인구 1명이 노인 1명 부양할 판,"""생산인구 1명이 노인 1명 부양할 판…“인구 축소는 상상 이상으로 고통스러운 ‘죽...",0,0,,0,0,daum,cafe
1,"""노인은 사람 아닌가""…'노 시니어 존' 등장에 찬반 가열",출처 : https://n.news.naver.com/article/001/0013...,0,195,혐오 대결로 이어지지 않게 자영업자 보호가 제대로 이루어지길\n기업이랑 비교하는 건...,4,0,daum,cafe
2,"노인부양 부담 커지는데..""65세 기준 올리자"" vs ""복지혜택 줄어""","노인부양 부담 커지는데..""65세 기준 올리자"" vs ""복지혜택 줄어""https:/...",0,0,,0,0,daum,cafe
3,노인학대 행위자 중 '배우자' 34.9%…노인부부 증가로 1년만에 5.8%p↑,출처 : 뉴스1 최현만 기자입력 2023. 6. 15. 11:03수정 2023. 6...,0,0,,8,0,daum,cafe
4,노인학대예방안내,노인학대예방안내 【노인 학대의 유형】 노인복지법 제1조의2와 제39조의9에 규정한...,0,4,이원옥 기자님~ 안녕하세요^^'노인 학대 예방수칙'이 도움이 되네요.\n잘보았습니다...,0,0,daum,cafe
...,...,...,...,...,...,...,...,...,...
175,노인은 무엇으로 사는가 - 감사의 마음으로 사는 것이다,[노인은 무엇으로 사는가? - 감사(感謝)의 마음으로 사는 것이다]\n\n이런 질문...,3,4,감사합니다\n좋은 글 잘 읽고 갑니다\n목요일날을 잘 보내셨는지요 저녁시간에 컴퓨터...,4,0,daum,cafe
176,한 장으로 보는 우리나라가 망한 이유.jpg,출처-대한민국 통계청 인류 역사상 유래 없는 현상 곧 우리나라 인구의 40%가 65...,3,59,통일되서 자신들의 기득권이 없어지느니 망해버리길 바랄겁니다. 그런자들 이니까\n인구...,2,0,daum,cafe
177,65세 이상 노인 혜택 50가지 몰라서 못 쓰는 사람이 더 많아(1탄),65세 이상 노인 혜택 50가지 몰라서 못 쓰는 사람이 더 많아(1탄)by M가이드...,0,0,,6,0,daum,cafe
178,"[청주주간보호 청주요양원 추천] kbs방영된 프로그램전문기관, 재활운동, 요양등급상...",KBS방영된 프로그램전문기관 9988 청주노인주간보호센터입니다. 33명 정원의 아늑...,0,0,,26,2,daum,cafe


In [11]:
# 데이터프레임을 CSV 파일로 저장
temp_file.to_csv("daum_cafe.csv", index = False, encoding="utf-8-sig")